In [1]:
from PyPDF2 import PdfFileReader

In [2]:
def get_info(path):
    with open(path, 'rb') as f:
        pdf = PdfFileReader(f)
        info = pdf.getDocumentInfo()
        number_of_pages = pdf.getNumPages()
    print(info)
    print(number_of_pages)
    author = info.author
    creator = info.creator
    producer = info.producer
    subject = info.subject
    title = info.title
if __name__ == '__main__':
    path = 'Professional Resume Format.pdf'
    get_info(path)

{'/CreationDate': "D:20140210124454-06'00'", '/Creator': 'Adobe InDesign CS6 (Windows)', '/ModDate': "D:20160225164221-06'00'", '/Producer': 'Adobe PDF Library 10.0.1', '/Trapped': '/False'}
13


In [8]:
def text_extractor(path):
    pdf_text = ""
    with open(path, 'rb') as f:
        pdf = PdfFileReader(f)
        number_of_pages = pdf.getNumPages()
        page = pdf.getPage(2)
        text = page.extractText()
        pagen = pdf.getPage(5)
        textn = pagen.extractText()
        pdf_text_2 = text.replace("\n", " ")
        pdf_textn = textn.replace("\n", " ")
        return pdf_text_2, pdf_textn
if __name__ == '__main__':
    path = 'Professional Resume Format.pdf'
text1, text2 = text_extractor(path)

In [9]:
text1

'Introduction to the  Professional Resume A wellŒcrafted, professional resume will open many doors and give you the best possible chance for getting an interview. Once you land the interview, then it™s up to you to land the job. So, resumes don™t get you the job, but  they are a critical element of the job search that leads to an offer. The staff at the Career Development Center at MTSU builds hundreds of resumes each year and also trains for resume writing by reviewing thousands of resumes written by professional resume writers.  We also listen to  feedback from recruiters and hiring managers.  Although you™ll find that just about everyone has a different opin-ion on how to write a resume, our goal is to help you create your own resume based on core principles so that  you™ll have a professional resume throughout the course of your career. So what exactly is a resume anyway? Let™s start with a basic definition. When you think about it, a resume is actually a marketing document that hi

In [10]:
text2

'FormatTake a look at the principles below to give your document effective format and design. Format has to do with the way your information is organized, while design deals with the many elements that will give your resume visual appeal. Format and design choices should be intentional and speak to your hiring audience. We like to use the word principles rather than rules because resume writing is a subjective process and many opinions exist. Our  advice is to always consider your hiring audience first and then utilize resume guides for tips and examples of how the professional resume is created.Principle #1 - Resume length. Consider limiting your resume to one page if you can describe your relevant and persuasive content within that frame. If you have a lot of content or you™re a seasoned professional with years of experience, you may need more than one page. What you don™t want is a partial page as this communicates an inability to be concise or that you lack content.Principle #2 - E